In [117]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFECV 
from sklearn.ensemble import RandomForestRegressor
from datetime import datetime
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [118]:
train=pd.read_csv("Train.csv")
test=pd.read_csv("Test.csv")

In [119]:
train.head()

,ID,DATOP,FLTID,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target
0,train_id_0,2016-01-03,TU 0712,CMN,TUN,2016-01-03 10:30:00,2016-01-03 12.55.00,ATA,TU 32AIMN,260.0
1,train_id_1,2016-01-13,TU 0757,MXP,TUN,2016-01-13 15:05:00,2016-01-13 16.55.00,ATA,TU 31BIMO,20.0
2,train_id_2,2016-01-16,TU 0214,TUN,IST,2016-01-16 04:10:00,2016-01-16 06.45.00,ATA,TU 32AIMN,0.0
3,train_id_3,2016-01-17,TU 0480,DJE,NTE,2016-01-17 14:10:00,2016-01-17 17.00.00,ATA,TU 736IOK,0.0
4,train_id_4,2016-01-17,TU 0338,TUN,ALG,2016-01-17 14:30:00,2016-01-17 15.50.00,ATA,TU 320IMU,22.0


In [120]:

train.isnull().sum()

ID        0
DATOP     0
FLTID     0
DEPSTN    0
ARRSTN    0
STD       0
STA       0
STATUS    0
AC        0
target    0
dtype: int64

In [121]:
#transform to date type
train['DATOP'] = pd.to_datetime(train['DATOP'])
test['DATOP'] = pd.to_datetime(test['DATOP'])

In [122]:
#transform id field
train['ID'] =train["ID"].str.split("_").str[-1].astype(int)
test['ID'] =test["ID"].str.split("_").str[-1].astype(int)

In [123]:
#extract day,month, and year
train['Year'] = train['DATOP'].dt.year
test['Year'] = test['DATOP'].dt.year

train['Month'] = train['DATOP'].dt.month
test['Month'] = test['DATOP'].dt.month

train['Day'] = train['DATOP'].dt.day
test['Day'] = test['DATOP'].dt.day

In [125]:
#add season feature
data = [train, test]
for dataset in data:
    dataset.loc[ (dataset['Month'] < 3) | (dataset['Month'] == 12), 'Season'] = 0
    dataset.loc[(dataset['Month'] >= 3) & (dataset['Month'] < 6), 'Season'] = 1
    dataset.loc[(dataset['Month'] >= 6) & (dataset['Month'] < 9), 'Season'] = 2
    dataset.loc[(dataset['Month'] >= 9) & (dataset['Month'] < 12), 'Season'] = 3
    dataset['Season'] = dataset['Season'].astype(int)

In [126]:
#transform scheduled time of departure
train['STD'] = pd.to_datetime(train['STD'])
test['STD'] = pd.to_datetime(test['STD'])

In [127]:
#transform scheduled time of arrival
train['STA']=train['STA'].astype(str).str.replace(".",":")
test['STA']=test['STA'].astype(str).str.replace(".",":")

train['STA'] = pd.to_datetime(train['STA'])
test['STA'] = pd.to_datetime(test['STA'])

C:\Users\LENOVO\AppData\Local\Temp/ipykernel_13948/2938100432.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  train['STA']=train['STA'].astype(str).str.replace(".",":")
C:\Users\LENOVO\AppData\Local\Temp/ipykernel_13948/2938100432.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  test['STA']=test['STA'].astype(str).str.replace(".",":")


In [128]:
#extract hours and minutes of departure and arrival separately
train['hour_of_departure'] = train['STD'].dt.hour 
test['hour_of_departure'] = test['STD'].dt.hour



train['hour_of_arrival'] = train['STA'].dt.hour 
test['hour_of_arrival'] = test['STA'].dt.hour

train['minute_of_departure'] = train['STD'].dt.minute 
test['minute_of_departure'] = test['STD'].dt.minute


train['minute_of_arrival'] = train['STA'].dt.minute 
test['minute_of_arrival'] = test['STA'].dt.minute



In [129]:
#calculate flight duration in minutes
train['flight_duration_minute'] = ((train['hour_of_arrival']*60) +train['minute_of_arrival'] ) - ((train['hour_of_departure']*60 )+train['minute_of_departure'])
test['flight_duration_minute'] = ((test['hour_of_arrival']*60) +test['minute_of_arrival'] ) - ((test['hour_of_departure']*60 )+test['minute_of_departure'])

In [130]:
#check for hour whether it's PM or AM
data = [train, test]
for dataset in data:
    dataset.loc[ (dataset['hour_of_departure'] < 12) , 'hour_of_departure_AM_PM'] = 0
    dataset.loc[(dataset['hour_of_departure'] >= 12) , 'hour_of_departure_AM_PM'] = 1
    dataset['hour_of_departure_AM_PM'] = dataset['hour_of_departure_AM_PM'].astype(int)
    
    

data = [train, test]
for dataset in data:
    dataset.loc[ (dataset['hour_of_arrival'] < 12) , 'hour_of_arrival_AM_PM'] = 0
    dataset.loc[(dataset['hour_of_arrival'] >= 12) , 'hour_of_arrival_AM_PM'] = 1
    dataset['hour_of_arrival_AM_PM'] = dataset['hour_of_arrival_AM_PM'].astype(int)

In [131]:
dateColumns=['DATOP','STD','STA']
for col in dateColumns:
  train[col] = pd.to_numeric(pd.to_datetime(train[col]))
  test[col] = pd.to_numeric(pd.to_datetime(test[col]))


In [132]:
train.drop('FLTID', inplace=True, axis=1)
test.drop('FLTID', inplace=True, axis=1)
categorical_features=train.select_dtypes(include=['object']).columns.tolist()


In [133]:
categorical_features

['DEPSTN', 'ARRSTN', 'STATUS', 'AC']

In [134]:
categorical_features.remove('STATUS')


In [135]:
one_hot_encoded_training_predictors = pd.get_dummies(train.STATUS)

In [136]:
one_hot_encoded_training_predictors

,ATA,DEL,DEP,RTR,SCH
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
107828,0,0,0,0,1
107829,0,0,0,0,1
107830,0,0,0,0,1
107831,1,0,0,0,0


In [137]:
one_hot_encoded_training_predictors_test = pd.get_dummies(test.STATUS)

In [138]:
one_hot_encoded_training_predictors_test

,ATA,DEL,DEP,RTR,SCH
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
...,...,...,...,...,...
9328,1,0,0,0,0
9329,0,0,0,0,1
9330,0,0,0,0,1
9331,0,0,0,0,1


In [139]:
train=pd.concat([train,one_hot_encoded_training_predictors],axis='columns')
train.drop(['STATUS'],axis='columns')
train

,ID,DATOP,DEPSTN,ARRSTN,STD,STA,STATUS,AC,target,Year,...,minute_of_departure,minute_of_arrival,flight_duration_minute,hour_of_departure_AM_PM,hour_of_arrival_AM_PM,ATA,DEL,DEP,RTR,SCH
0,0,1451779200000000000,CMN,TUN,1451817000000000000,1451825700000000000,ATA,TU 32AIMN,260.0,2016,...,30,55,145,0,1,1,0,0,0,0
1,1,1452643200000000000,MXP,TUN,1452697500000000000,1452704100000000000,ATA,TU 31BIMO,20.0,2016,...,5,55,110,1,1,1,0,0,0,0
2,2,1452902400000000000,TUN,IST,1452917400000000000,1452926700000000000,ATA,TU 32AIMN,0.0,2016,...,10,45,155,0,0,1,0,0,0,0
3,3,1452988800000000000,DJE,NTE,1453039800000000000,1453050000000000000,ATA,TU 736IOK,0.0,2016,...,10,0,170,1,1,1,0,0,0,0
4,4,1452988800000000000,TUN,ALG,1453041000000000000,1453045800000000000,ATA,TU 320IMU,22.0,2016,...,30,50,80,1,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107828,107828,1530748800000000000,TUN,TUN,1530831600000000000,1530842400000000000,SCH,TU 32AIML,0.0,2018,...,0,0,-1260,1,0,0,0,0,0,1
107829,107829,1515801600000000000,DJE,TUN,1515830400000000000,1515834000000000000,SCH,UG AT7AT7,0.0,2018,...,0,0,60,0,0,0,0,0,0,1
107830,107830,1541548800000000000,TUN,TUN,1541566800000000000,1541595000000000000,SCH,TU 736IOK,0.0,2018,...,0,50,470,0,1,0,0,0,0,1
107831,107831,1516665600000000000,TUN,DJE,1516730400000000000,1516733100000000000,ATA,TU CR9ISA,0.0,2018,...,0,45,45,1,1,1,0,0,0,0


In [140]:
test=pd.concat([test,one_hot_encoded_training_predictors_test],axis='columns')
test.drop(['STATUS'],axis='columns')
test

,ID,DATOP,DEPSTN,ARRSTN,STD,STA,STATUS,AC,Year,Month,...,minute_of_departure,minute_of_arrival,flight_duration_minute,hour_of_departure_AM_PM,hour_of_arrival_AM_PM,ATA,DEL,DEP,RTR,SCH
0,0,1462320000000000000,DJE,TUN,1462344000000000000,1462347000000000000,ATA,TU 32AIMF,2016,5,...,40,30,50,0,0,1,0,0,0,0
1,1,1462406400000000000,TUN,BKO,1462461600000000000,1462478700000000000,ATA,TU 320IMW,2016,5,...,20,5,285,1,1,1,0,0,0,0
2,2,1462492800000000000,FRA,TUN,1462528800000000000,1462537500000000000,ATA,TU 32AIMC,2016,5,...,0,25,145,0,1,1,0,0,0,0
3,3,1462924800000000000,BEY,TUN,1462959600000000000,1462972200000000000,ATA,TU 31BIMO,2016,5,...,40,10,210,0,1,1,0,0,0,0
4,4,1462924800000000000,ORY,MIR,1462960200000000000,1462970100000000000,ATA,TU 736IOQ,2016,5,...,50,35,165,0,1,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9328,9328,1536710400000000000,TUN,NCE,1536761700000000000,1536767100000000000,ATA,TU 320IMV,2018,9,...,15,45,90,1,1,1,0,0,0,0
9329,9329,1538006400000000000,TUN,TUN,1538085600000000000,1538096400000000000,SCH,TU 32AIMG,2018,9,...,0,0,-1260,1,0,0,0,0,0,1
9330,9330,1535932800000000000,SJJ,TUN,1535966400000000000,1535973000000000000,SCH,TU CR9ISA,2018,9,...,20,10,110,0,0,0,0,0,0,1
9331,9331,1536969600000000000,TUN,DJE,1537021800000000000,1537025400000000000,SCH,UG AT7LBD,2018,9,...,30,30,60,1,1,0,0,0,0,1


In [150]:
####DELETING outliers in AC column
AC_counts = train['AC'].value_counts().to_dict()
"print(counts)"
#make 1500 as threshold....
drop_AC=[]
for key,value in AC_counts.items():
  if value<1500:
    drop_AC.append(key)
####DELETING outliers in ARRSTN column
ARRSTN_counts = train['ARRSTN'].value_counts().to_dict()
"print(counts)"
#make 1500 as threshold....
drop_ARRSTN=[]
for key,value in ARRSTN_counts.items():
  if value<500:
    drop_ARRSTN.append(key)
    
####DELETING outliers in DEPSTN column
DEPSTN_counts = train['DEPSTN'].value_counts().to_dict()
"print(counts)"
#make 1500 as threshold....
drop_DEPSTN=[]
for key,value in DEPSTN_counts.items():
  if value<500:
    drop_DEPSTN.append(key)
    
####DELETING outliers in STATUS column
STATUS_counts = train['STATUS'].value_counts().to_dict()
"print(counts)"
#make 1500 as threshold....
drop_STATUS=[]
for key,value in STATUS_counts.items():
  if value<400:
    drop_STATUS.append(key)

In [151]:
print(train.shape)
print(test.shape)

(84276, 28)
(7082, 27)


In [152]:
train=train[~train.AC.isin(drop_AC)]
test=test[~test.AC.isin(drop_AC)]
train=train[~train.ARRSTN.isin(drop_ARRSTN)]
test=test[~test.ARRSTN.isin(drop_ARRSTN)]
train=train[~train.DEPSTN.isin(drop_DEPSTN)]
test=test[~test.DEPSTN.isin(drop_DEPSTN)]
train=train[~train.STATUS.isin(drop_STATUS)]
test=test[~test.STATUS.isin(drop_STATUS)]


In [153]:
print(train.shape)
print(test.shape)

(81527, 28)
(6875, 27)


In [154]:
for cat_col in categorical_features:
  enc_train = (train.groupby(f'{cat_col}').size()) / len(train)
  train[f'{cat_col}_encode'] = train[f'{cat_col}'].apply(lambda x : enc_train[x])
  enc_test = (test.groupby(f'{cat_col}').size()) / len(test)
  test[f'{cat_col}_encode'] = test[f'{cat_col}'].apply(lambda x : enc_test[x])


In [155]:
#training
valid_columns=['DATOP', 'STD', 'STA','ARRSTN_encode','ATA','DEL','DEP','RTR','SCH',	'AC_encode']
X = train[valid_columns]
Y = train['target']
x_test = test[valid_columns]
x_train, x_val, y_train, y_val  = train_test_split(X, Y ,test_size=0.20 ,random_state=17 )

In [156]:
model = RandomForestRegressor(random_state=42,bootstrap=True,max_depth=None, max_features= 'log2', min_samples_split= 8, n_estimators=350)
model.fit(x_train, y_train);
predictions = model.predict(x_val)
from sklearn.metrics import mean_squared_error
"from sklearn.metrics import mean_absolute_error"

from math import sqrt
"mae=mean_absolute_error(y_train,predictions)"
rmse = sqrt(mean_squared_error(y_val,predictions))
"print(mae)"
print(rmse)

95.9549813989998
